<a href="https://colab.research.google.com/github/alfa7g7/Analitica-prescriptiva/blob/main/Talleres/Taller_2_Analitica_Prescriptiva_Prog_ent_mixta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TALLER 2 | ANALITICA PRESCRIPTIVA | PULP | PROGRAMACION ENTERA MIXTA

### INTEGRANTES:
              Arlex Pino
              Raul Echeverry
              Esteban Ordoñez
              Fabian Salazar Figueroa

In [ ]:
# Instalando libreria para poder modelar
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 33.9 MB/s eta 0:00:00


### **1.	General Wheels Co.**

El consejo directivo de la General Wheels Co. Está considerando siete grandes inversiones de capital. Estas inversiones difieren en la utilidad estimada a largo plazo (valor presente neto) que van a generar, así como en la cantidad de capital requerido, como se muestra en la siguiente tabla (en millones de dólares):

|Oportunidad de inversion|Utilidad estimada|Capital requerido|
|:-:|:-:|:-:|
|1|9|25|
|2|16|40|
|3|18|45|
|4|15|35|
|5|6|15|
|6|12|30|
|7|11|25|



La cantidad total de capital disponible para estas inversiones es de US$100.000.000. Las oportunidades de inversión 1 y 2 son mutuamente excluyentes y también lo son la 3 y 4. Además, no pueden intentarse 3 ni 4 a menos que se decida invertir en 1 o en 2. No existen restricciones sobre las oportunidades 5, 6 y 7. El objetivo es seleccionar la combinación de inversiones de capital que maximice la utilidad total estimada a largo plazo (valor presente neto).

Formular y resolver en PULP ó SOLVER un modelo de programación lineal binaria para este problema

In [ ]:
import pulp

# Problema 1: Inversiones de General Wheels Co.
# Definir el problema
problema1 = pulp.LpProblem("Maximizar_Utilidad", pulp.LpMaximize)

# Definir variables binarias
x = {i: pulp.LpVariable(f"x{i}", cat='Binary') for i in range(1, 8)}

# Función objetivo: Maximizar la utilidad estimada
utilidad = {1: 9, 2: 16, 3: 18, 4: 15, 5: 6, 6: 12, 7: 11}  # En millones
problema1 += pulp.lpSum(utilidad[i] * x[i] for i in range(1, 8)), "Total_Utilidad"

# Restricción de presupuesto
costos = {1: 25, 2: 40, 3: 45, 4: 35, 5: 15, 6: 30, 7: 25}  # En millones
problema1 += pulp.lpSum(costos[i] * x[i] for i in range(1, 8)) <= 100, "Presupuesto"

# Restricciones de exclusión mutua
problema1 += x[1] + x[2] <= 1, "Mutua_1_2"
problema1 += x[3] + x[4] <= 1, "Mutua_3_4"

# Restricción de dependencia
problema1 += x[3] <= x[1] + x[2], "Dependencia_3"
problema1 += x[4] <= x[1] + x[2], "Dependencia_4"

# Resolver
problema1.solve()

# Obtener resultados
inversiones_seleccionadas = [i for i in range(1, 8) if pulp.value(x[i]) == 1]
utilidad_total = pulp.value(problema1.objective)

# Imprimir resultados
for i in range(1, 8):
    print(f"Inversión {i}: {pulp.value(x[i])}")
print(f"Utilidad Total Máxima: {pulp.value(problema1.objective)} millones")

# Imprimir resultados A usuario final
print("\nInversiones seleccionadas:", inversiones_seleccionadas)
print(f"Utilidad Total Máxima: {utilidad_total} millones")

# Conclusión
print("\nConclusión:")
print(f"La mejor combinación de inversiones dentro del presupuesto de 100 millones es {inversiones_seleccionadas}, logrando una utilidad total máxima de {utilidad_total} millones.")


Inversión 1: 0.0
Inversión 2: 1.0
Inversión 3: 0.0
Inversión 4: 1.0
Inversión 5: 0.0
Inversión 6: 0.0
Inversión 7: 1.0
Utilidad Total Máxima: 42.0 millones

Inversiones seleccionadas: [2, 4, 7]
Utilidad Total Máxima: 42.0 millones

Conclusión:
La mejor combinación de inversiones dentro del presupuesto de 100 millones es [2, 4, 7], logrando una utilidad total máxima de 42.0 millones.


### **2. Series de Televisión**

Su amigo que es fanático de las series de televisión le ha pedido ayuda para tomar complejas decisiones. La decisión está basada en dos aspectos, primero que serie comprar y segundo cuantas horas de la semana “invertir” viendo la serie. El costo de oportunidad de la hora viendo televisión es diferente dado que hay series que exigen menos concentración y se pueden desarrollar actividades adicionales de tal forma que no se “pierda” totalmente ese tiempo. Por otro lado, su amigo dispone de 20 horas para ser asignadas a ratos de esparcimiento en sus series favoritas. Además, para poder dormir bien es importante balancear el tipo de series que se ven, en ese orden de ideas no se pueden ver las series “Dahmer” y “Stranger Things” en el mismo periodo de tiempo. De igual forma para asegurar variedad en la recomendación, no se pueden asignar más de 8 horas a una misma serie. Su amigo sabe que Ud. está viendo el curso de investigación de operaciones y le ha pedido que le ayude a formular un modelo que le permita invertir bien su tiempo libre minimizando el costo

|Serie|Costo|Costo de oportunidad hora|
|:-|:-|:-|
|House of the dragon|30000|1200|
|Peaky Blinders|35000|7000|
|Homeland|20000|8000|
|Dahmer|25000|10000|
|Breaking Bad|20000|9000|
|Stranger Things|15000|7000|
|Dexter|40000|1000|


In [ ]:
import pulp

# Problema 2: Selección de series
# Definir el problema
problema2 = pulp.LpProblem("Minimizar_Costo_Total", pulp.LpMinimize)

# Definir variables de decisión
series = ["House of the dragon", "Peaky Blinders", "Homeland", "Dahmer", "Breaking Bad", "Stranger Things", "Dexter"]

# Variables binarias para indicar si se compra la serie
compra = {s: pulp.LpVariable(f"compra_{s.replace(' ', '_')}", cat='Binary') for s in series}

# Variables continuas para asignar horas a cada serie
horas = {s: pulp.LpVariable(f"h_{s.replace(' ', '_')}", lowBound=0, upBound=8, cat='Continuous') for s in series}

# Costos de oportunidad por hora
costo_oportunidad = {
    "House of the dragon": 1200,
    "Peaky Blinders": 7000,
    "Homeland": 8000,
    "Dahmer": 10000,
    "Breaking Bad": 9000,
    "Stranger Things": 7000,
    "Dexter": 1000
}

# Costos de compra por serie
costo_compra = {
    "House of the dragon": 30000,
    "Peaky Blinders": 35000,
    "Homeland": 20000,
    "Dahmer": 25000,
    "Breaking Bad": 20000,
    "Stranger Things": 15000,
    "Dexter": 40000
}

# Función objetivo: Minimizar el costo total (compra + oportunidad)
problema2 += pulp.lpSum((costo_oportunidad[s] * horas[s]) + (costo_compra[s] * compra[s]) for s in series), "Total_Costo"

# Restricción de tiempo total disponible
problema2 += pulp.lpSum(horas[s] for s in series) == 20, "Tiempo_Exacto"

# Relación entre compra y horas asignadas (solo se pueden asignar horas si la serie es comprada)
for s in series:
    problema2 += horas[s] <= 8 * compra[s], f"Horas_si_Compra_{s}"

# Restricción de exclusión mutua entre "Dahmer" y "Stranger Things"
problema2 += horas["Dahmer"] + horas["Stranger Things"] <= 8, "Mutua_Dahmer_StrangerThings"

# Resolver
problema2.solve()

# Obtener resultados
horas_asignadas = {s: pulp.value(horas[s]) for s in series if pulp.value(horas[s]) > 0}
series_compradas = [s for s in series if pulp.value(compra[s]) > 0.5]
costo_total = pulp.value(problema2.objective)

# Verificar valores de las variables
print("Series compradas:")
print(series_compradas)

print("\nHoras asignadas por serie:")
for s in series:
    print(f"{s}: {pulp.value(horas[s])}")

# Imprimir resultados
print("\nCosto total mínimo (compra + oportunidad):", costo_total)

# Conclusión
print("\nConclusión:")
print(f"La mejor selección minimiza el costo total (compra + oportunidad) a {costo_total}. Las series compradas son: {series_compradas}. La asignación de horas por serie es: {horas_asignadas}.")


Series compradas:
['House of the dragon', 'Stranger Things', 'Dexter']

Horas asignadas por serie:
House of the dragon: 8.0
Peaky Blinders: 0.0
Homeland: 0.0
Dahmer: 0.0
Breaking Bad: 0.0
Stranger Things: 4.0
Dexter: 8.0

Costo total mínimo (compra + oportunidad): 130600.0

Conclusión:
La mejor selección minimiza el costo total (compra + oportunidad) a 130600.0. Las series compradas son: ['House of the dragon', 'Stranger Things', 'Dexter']. La asignación de horas por serie es: {'House of the dragon': 8.0, 'Stranger Things': 4.0, 'Dexter': 8.0}.


### **3. Gotham city Ambulance problem**

Gotham City has been divided into eight districts. The time (in minutes) it takes an ambulance to travel from one district to another is shown in Table 100. The population of each district (in thousands) is as follows: district 1, 40; district 2, 30; district 3,  35; district 4, 20; district 5, 15; district 6, 50; district 7, 45; district 8, 60. The city has only two ambulances and wants to locate them to maximize the number of people who live within 2 minutes of an ambulance. Formulate an IP to accomplish this goal.

TABLE 100 - District

|District|1|2|3|4|5|6|7|8|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|1|0|3|4|6|8|9|8|10|
|2|3|0|5|4|8|6|12|9|
|3|4|5|0|2|2|3|5|7|
|4|6|4|2|0|3|2|5|4|
|5|8|8|2|3|0|2|2|4|
|6|9|6|3|2|2|0|3|2|
|7|8|12|5|5|2|3|0|2|
|8|10|9|7|4|4|2|2|0|

In [ ]:
import pulp

# Definir el problema
problema3 = pulp.LpProblem("Maximizar_Cobertura_Ambulancias", pulp.LpMaximize)

# Distritos y poblaciones
distritos = list(range(1, 9))
poblacion = {1: 40, 2: 30, 3: 35, 4: 20, 5: 15, 6: 50, 7: 45, 8: 60} # POblaciones en Miles

# Matriz de tiempos de viaje
tiempos = {
    1: {1: 0, 2: 3, 3: 4, 4: 6, 5: 8, 6: 9, 7: 8, 8: 10},
    2: {1: 3, 2: 0, 3: 5, 4: 4, 5: 8, 6: 6, 7: 12, 8: 9},
    3: {1: 4, 2: 5, 3: 0, 4: 2, 5: 2, 6: 3, 7: 5, 8: 7},
    4: {1: 6, 2: 4, 3: 2, 4: 0, 5: 3, 6: 2, 7: 5, 8: 4},
    5: {1: 8, 2: 8, 3: 2, 4: 3, 5: 0, 6: 2, 7: 2, 8: 4},
    6: {1: 9, 2: 6, 3: 3, 4: 2, 5: 2, 6: 0, 7: 3, 8: 2},
    7: {1: 8, 2: 12, 3: 5, 4: 5, 5: 2, 6: 3, 7: 0, 8: 2},
    8: {1: 10, 2: 9, 3: 7, 4: 4, 5: 4, 6: 2, 7: 2, 8: 0}
}

# Variables binarias para la ubicación de ambulancias
x = {i: pulp.LpVariable(f"x_{i}", cat='Binary') for i in distritos}

# Variables binarias para indicar si un distrito está cubierto
y = {i: pulp.LpVariable(f"y_{i}", cat='Binary') for i in distritos}

# Función objetivo: Maximizar la población cubierta
problema3 += pulp.lpSum(poblacion[i] * y[i] for i in distritos), "Poblacion_Cubierta"

# Restricción: Solo se pueden ubicar dos ambulancias
problema3 += pulp.lpSum(x[i] for i in distritos) == 2, "Cantidad_Ambulancias"

# Restricciones de cobertura (distritos dentro de 2 minutos)
for i in distritos:
    problema3 += y[i] <= pulp.lpSum(x[j] for j in distritos if tiempos[i][j] <= 2), f"Cobertura_{i}"

# Resolver el problema
problema3.solve()

# Resultados
ambulancias_ubicadas = [i for i in distritos if pulp.value(x[i]) == 1]
distritos_cubiertos = [i for i in distritos if pulp.value(y[i]) == 1]
poblacion_total_cubierta = pulp.value(problema3.objective)

print("Ubicación de ambulancias en distritos:", ambulancias_ubicadas)
print("\nDistritos cubiertos:", distritos_cubiertos)
print("\nPoblación total cubierta:", poblacion_total_cubierta)

# Conclusión
echo = f"Se deben colocar ambulancias en los distritos {ambulancias_ubicadas}, asegurando cobertura para los distritos {distritos_cubiertos}, cubriendo en total {poblacion_total_cubierta} mil personas."
print("\nConclusión:", echo)


Ubicación de ambulancias en distritos: [5, 6]

Distritos cubiertos: [3, 4, 5, 6, 7, 8]

Población total cubierta: 225.0

Conclusión: Se deben colocar ambulancias en los distritos [5, 6], asegurando cobertura para los distritos [3, 4, 5, 6, 7, 8], cubriendo en total 225.0 mil personas.


### **4. Comité quejas Universidad Arkasas**

La Universidad de Arkansas va a formar un comité para atender las quejas de los estudiantes. La administración desea que el comité incluya al menos una mujer, un hombre, un estudiante, un administrador y un profesor. Diez personas (identificadas, por simplicidad, con las letras de la a a la j) han sido nominadas, y se les ha combinado en las distintas categorías siguientes:



|Categoria|Personas|
|:-|:-|
|Mujeres|a, b, c, d, e|
|Hombres|f, g, h, i, j|
|Estudiantes|a, b, c, j|
|Administradores|e, f|
|Profesores|d, g, h, i|


La Universidad de Arkansas desea formar el menor comité con la representación de cada una de las cinco categorías. Formule el problema como un PLE, y halle la solución óptima.

In [ ]:
import pulp

# Definir el problema de optimización
problema4 = pulp.LpProblem("Minimizar_Tamaño_Comité", pulp.LpMinimize)

# Definir las personas
personas = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]

# Variables binarias (1 si la persona es seleccionada, 0 si no)
x = {p: pulp.LpVariable(f"x_{p}", cat='Binary') for p in personas}

# Función objetivo: minimizar el número total de personas en el comité
problema4 += pulp.lpSum(x[p] for p in personas), "Minimizar_Tamaño_Comité"

# Restricciones de representación
problema4 += pulp.lpSum(x[p] for p in ["a", "b", "c", "d", "e"]) >= 1, "AlMenos_Una_Mujer"
problema4 += pulp.lpSum(x[p] for p in ["f", "g", "h", "i", "j"]) >= 1, "AlMenos_Un_Hombre"
problema4 += pulp.lpSum(x[p] for p in ["a", "b", "c", "j"]) >= 1, "AlMenos_Un_Estudiante"
problema4 += pulp.lpSum(x[p] for p in ["e", "f"]) >= 1, "AlMenos_Un_Administrador"
problema4 += pulp.lpSum(x[p] for p in ["d", "g", "h", "i"]) >= 1, "AlMenos_Un_Profesor"

# Resolver el problema
problema4.solve()

# Obtener los resultados
personas_seleccionadas = [p for p in personas if pulp.value(x[p]) > 0.5]
tamano_comite = pulp.value(problema4.objective)

# Imprimir resultados
print("Personas seleccionadas para el comité:")
print(personas_seleccionadas)
print("\nTamaño mínimo del comité:", int(tamano_comite))

# Conclusión
print("\nConclusión:")
print(f"La mejor selección de personas para el comité son: {personas_seleccionadas}. De esta manera se cumple con todos los requisitos de la universidad y se tendría un comite con {tamano_comite} miembros.")


Personas seleccionadas para el comité:
['d', 'f', 'j']

Tamaño mínimo del comité: 3

Conclusión:
La mejor selección de personas para el comité son: ['d', 'f', 'j']. De esta manera se cumple con todos los requisitos de la universidad y se tendría un comite con 3.0 miembros.


### **5. Oil delivery truck**

A Sunco oil delivery truck contains five compartments, holding up to 2,700, 2,800, 1,100, 1,800, and 3,400 gallons of fuel, respectively. The company must deliver three types of fuel (super, regular, and unleaded) to a customer. The demands, penalty per gallon short, and the maximum allowed shortage are given in Table 29. Each compartment of the truck can carry only one type of gasoline. Formulate an IP whose solution will tell Sunco how to load the truck in a way that minimizes shortage costs

TABLE 29

|Type of gasoline|Demand|Cost per gallon short ($)|Maximum allowed shortage|
|:-:|:-:|:-:|:-:|
|Super|2900|10|500|
|Regular|4000|8|500|
|Unleaded|4900|6|500|



In [ ]:
import pulp

# Definir el problema
problema5 = pulp.LpProblem("Minimizar_Costo_Escasez", pulp.LpMinimize)

# Definir variables de decisión
combustibles = ["Super", "Regular", "Unleaded"]
compartimentos = ["C1", "C2", "C3", "C4", "C5"]
capacidad = {"C1": 2700, "C2": 2800, "C3": 1100, "C4": 1800, "C5": 3400}
demanda = {"Super": 2900, "Regular": 4000, "Unleaded": 4900}
costo_escasez = {"Super": 10, "Regular": 8, "Unleaded": 6}
max_escasez = 500

# Variables de cantidad de combustible asignado a cada compartimento
x = {(c, f): pulp.LpVariable(f"x_{c}_{f}", lowBound=0, upBound=capacidad[c], cat='Continuous')
     for c in compartimentos for f in combustibles}

# Variables de escasez de combustible
escasez = {f: pulp.LpVariable(f"escasez_{f}", lowBound=0, upBound=max_escasez, cat='Continuous')
           for f in combustibles}

# Variables binarias para indicar qué tipo de combustible va en cada compartimento
y = {(c, f): pulp.LpVariable(f"y_{c}_{f}", cat='Binary') for c in compartimentos for f in combustibles}

# Función objetivo: Minimizar el costo total de escasez
problema5 += pulp.lpSum(costo_escasez[f] * escasez[f] for f in combustibles), "Costo_Total_Escasez"

# Restricción de demanda (asignaciones más escasez deben cubrir la demanda)
for f in combustibles:
    problema5 += pulp.lpSum(x[c, f] for c in compartimentos) + escasez[f] == demanda[f], f"Demanda_{f}"

# Restricción de unicidad: cada compartimento solo puede llevar un tipo de combustible
for c in compartimentos:
    problema5 += pulp.lpSum(y[c, f] for f in combustibles) == 1, f"Unicidad_{c}"
    for f in combustibles:
        problema5 += x[c, f] <= capacidad[c] * y[c, f], f"Capacidad_{c}_{f}"

# Resolver el problema
problema5.solve()

# Obtener resultados
asignacion_combustible = {(c, f): pulp.value(x[c, f]) for c in compartimentos for f in combustibles if pulp.value(x[c, f]) > 0}
escasez_total = {f: pulp.value(escasez[f]) for f in combustibles}
costo_total = pulp.value(problema5.objective)

# Imprimir resultados
print("Asignación de combustible en cada compartimento:")
for (c, f), cantidad in asignacion_combustible.items():
    print(f"Compartimento {c} -> {f}: {cantidad} galones")

print("\nEscasez total por tipo de combustible:")
for f, cantidad in escasez_total.items():
    print(f"{f}: {cantidad} galones")

print("\nCosto total de escasez:", costo_total)

# Conclusión
print("\nConclusión:")
print(f"La mejor distribución minimiza el costo total de escasez a {costo_total}. La asignación de combustible por compartimento es: {asignacion_combustible}. La escasez final por tipo de combustible es: {escasez_total}.")


Asignación de combustible en cada compartimento:
Compartimento C1 -> Regular: 2700.0 galones
Compartimento C2 -> Super: 2800.0 galones
Compartimento C3 -> Regular: 1100.0 galones
Compartimento C4 -> Unleaded: 1500.0 galones
Compartimento C5 -> Unleaded: 3400.0 galones

Escasez total por tipo de combustible:
Super: 100.0 galones
Regular: 200.0 galones
Unleaded: 0.0 galones

Costo total de escasez: 2600.0

Conclusión:
La mejor distribución minimiza el costo total de escasez a 2600.0. La asignación de combustible por compartimento es: {('C1', 'Regular'): 2700.0, ('C2', 'Super'): 2800.0, ('C3', 'Regular'): 1100.0, ('C4', 'Unleaded'): 1500.0, ('C5', 'Unleaded'): 3400.0}. La escasez final por tipo de combustible es: {'Super': 100.0, 'Regular': 200.0, 'Unleaded': 0.0}.
